v2: loại bỏ phiên những record > 1 

In [1]:
import numpy as np
import faiss   

In [2]:
import os
import time
name = "v2-Faiss-EM-TED-bert-en-vi-0.999-"+ str(time.strftime("%Y%m%d-%H%M"))

!mkdir $name
path = os.getcwd() + "/" + name
os.chdir(path)

!pwd


os.chdir(path)


# import os
# os.chdir("/home/lw/clc_fairseq/TED-20210121-1335")

/home/chinh/clc_data/Script/v2-Faiss-EM-TED-bert-en-vi-0.999-20210401-1034


In [3]:
!wget https://github.com/hoangtrungchinh/clc_data/raw/master/en_vi_iwslt.tar.gz
!mkdir data_bin
!tar -xvf  'en_vi_iwslt.tar.gz' -C 'data_bin'

--2021-04-01 10:34:54--  https://github.com/hoangtrungchinh/clc_data/raw/master/en_vi_iwslt.tar.gz
Resolving github.com (github.com)... 13.229.188.59
Connecting to github.com (github.com)|13.229.188.59|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/en_vi_iwslt.tar.gz [following]
--2021-04-01 10:34:55--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/en_vi_iwslt.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9044100 (8,6M) [application/octet-stream]
Saving to: ‘en_vi_iwslt.tar.gz’

en_vi_iwslt.tar.gz  100%[===================>]   8,62M  6,23MB/s    in 1,4s    

2021-04-01 10:34:57 (6,23 MB/s) - ‘en_vi_iwslt.tar.gz’ saved [9044100/904410

In [4]:
# import file
import numpy as np
import glob

en_files = sorted(glob.glob("data_bin/*.en"))
vi_files = sorted(glob.glob("data_bin/*.vi"))
print(en_files)
print(vi_files)

lst_en = []
lst_vi = []

start = time.time()

for file in en_files:
  with open(file) as file_in:
    for line in file_in:
      lst_en.append(line)

for file in vi_files:
  with open(file) as file_in:
    for line in file_in:
      lst_vi.append(line)

print(len(lst_en))
print(len(lst_vi))

['data_bin/IWSLT15.TED.tst2012.en-vi.en', 'data_bin/IWSLT15.TED.tst2013.en-vi.en', 'data_bin/IWSLT15.TED.tst2015.en-vi.en', 'data_bin/train.en']
['data_bin/IWSLT15.TED.tst2012.en-vi.vi', 'data_bin/IWSLT15.TED.tst2013.en-vi.vi', 'data_bin/IWSLT15.TED.tst2015.en-vi.vi', 'data_bin/train.vi']
120956
120956


In [5]:
# SPLIT TRAIN, TEST, VALID
from sklearn.model_selection import train_test_split
en_train, en_test_valid, vi_train, vi_test_valid =  train_test_split(lst_en, lst_vi, test_size=0.2, random_state=123)
en_valid, en_test, vi_valid, vi_test = train_test_split(en_test_valid, vi_test_valid, test_size=0.5, random_state=123)
print(len(en_train))
print(len(vi_train))
print(len(en_valid))
print(len(vi_valid))
print(len(en_test))
print(len(vi_test))
print(en_train[20])
print(vi_train[20])
print(en_valid[-1])
print(vi_valid[-1])
print(en_test[-1])
print(vi_test[-1])

96764
96764
12096
12096
12096
12096
And , in fact , when we did some experiments looking at this , what we found is that 46 percent of the variation in how many friends you have is explained by your genes .

Trong thực tế , khi thực hiện một vài thí nghiệm nghiên cứu vấn đề này , chúng tôi nhận ra 46 % sự khác biệt trong số bạn bè của bạn được giải thích bằng gen của bạn .

This is a chart of what it looked like when it first became popular last summer .

Đây là biểu đồ thể hiện điều đó khi video này lần đầu tiên trở nên phổ biến vào mùa hè trước .

And what will happen in the future ?

Chuyện gì sẽ xảy ra trong tương lai ?



In [6]:
file_variable = [en_train,vi_train,en_valid,vi_valid,en_test,vi_test]
file_names = ["en_train","vi_train","en_valid","vi_valid","en_test","vi_test"]

for i in range(len(file_names)):
  with open(file_names[i], 'w') as f:
    for item in file_variable[i]:
      f.write(item)
!ls -al

total 36844
drwxrwxr-x  3 chinh chinh     4096 Thg 4   1 10:34 .
drwxrwxr-x 10 chinh chinh     4096 Thg 4   1 10:34 ..
drwxrwxr-x  2 chinh chinh     4096 Thg 4   1 10:34 data_bin
-rw-rw-r--  1 chinh chinh  1260181 Thg 4   1 10:34 en_test
-rw-rw-r--  1 chinh chinh 10011509 Thg 4   1 10:34 en_train
-rw-rw-r--  1 chinh chinh  1249342 Thg 4   1 10:34 en_valid
-rw-rw-r--  1 chinh chinh  9044100 Thg 4   1 10:34 en_vi_iwslt.tar.gz
-rw-rw-r--  1 chinh chinh  1624771 Thg 4   1 10:34 vi_test
-rw-rw-r--  1 chinh chinh 12896000 Thg 4   1 10:34 vi_train
-rw-rw-r--  1 chinh chinh  1609835 Thg 4   1 10:34 vi_valid


In [7]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('stsb-bert-large')

In [8]:
start = time.time()
srcfile = "en_train"
tarfile = "vi_train"
lst_srcfile = en_train
lst_tarfile = vi_train

vec_lst_srcfile = model.encode(lst_srcfile)
print('=== Ending, Total time (second): ', time.time() - start)

=== Ending, Total time (second):  1092.1983361244202


Test Cosin similarity

In [9]:
# import numpy as np
# from sklearn.preprocessing import normalize
# eq1=gt0=gt1=eq0=sm0=0
# tmp_arr = []
# arr_gt_1 = []
# for i in range(len(vec_lst_srcfile)):
#     x = vec_lst_srcfile[i]
#     norm = np.linalg.norm(x)
#     if norm == 0:
#         eq0+=1
#     elif norm ==1:
#         eq1+=1
#     elif norm>0 and norm < 1:
#         gt0+=1
#     elif norm >1:
#         arr_gt_1.append(i)
#         gt1+=1
#     elif norm <=0:
#         sm0+=1
#     temp = x / norm
        
#     tmp_arr.append(temp)
    
# vec_lst_srcfile_norm = np.array(tmp_arr)
# print(eq0, eq1, gt0, gt1, sm0)
# print(np.sum([eq0, eq1, gt0, gt1, sm0]))

# arr_gt_1_norm = []
# eq1=gt0=gt1=eq0=sm0=0
# for i in range(len(vec_lst_srcfile_norm)):
#     x = vec_lst_srcfile_norm[i]
#     norm = np.linalg.norm(x)
#     if norm == 0:
#         eq0+=1
#     elif norm ==1:
#         eq1+=1
#     elif norm>0 and norm < 1:
#         gt0+=1
#     elif norm >1:
#         arr_gt_1_norm.append(i)
#         gt1+=1
#     elif norm <=0:
#         sm0+=1

# print(eq0, eq1, gt0, gt1, sm0)

# print(np.sum([eq0, eq1, gt0, gt1, sm0]))
# print(arr_gt_1[:100])
# print(arr_gt_1_norm[:10])

In [10]:
import faiss
start = time.time()
index = faiss.IndexFlatL2(vec_lst_srcfile.shape[1])
index.ntotal
faiss.normalize_L2(vec_lst_srcfile)
index.add(vec_lst_srcfile)
k = 6
distance, index = index.search(vec_lst_srcfile, k)

print('=== Ending, Total time (second): ', time.time() - start)
print(len(distance))
print(len(index))

print(index)
print(distance)

=== Ending, Total time (second):  48.70456409454346
96764
96764
[[    0 35305 68662 43042  4864 48699]
 [    1 42253 63758 68947 19399  1353]
 [    2 86284 44936 11949 67241 60728]
 ...
 [96761 14876 41844 72724 32947 67302]
 [96762 77262 49083 94942 31069 44234]
 [96763 64661 49266   643 65358 53288]]
[[2.3841858e-07 4.2096364e-01 5.7486677e-01 5.9295475e-01 7.6007748e-01
  7.8514624e-01]
 [2.3841858e-07 6.4205766e-01 6.4899182e-01 6.7520106e-01 6.9696569e-01
  7.0406759e-01]
 [1.1920929e-07 8.7000871e-01 9.1394186e-01 9.8367453e-01 9.8373234e-01
  9.9183512e-01]
 ...
 [0.0000000e+00 8.1287539e-01 8.6379337e-01 9.1680264e-01 9.4501078e-01
  9.4541097e-01]
 [2.3841858e-07 7.5868464e-01 8.3096635e-01 9.0191162e-01 9.0270054e-01
  9.0406740e-01]
 [0.0000000e+00 5.8189964e-01 6.0680532e-01 6.5279508e-01 6.8669844e-01
  6.9454706e-01]]


In [11]:
i = 231
j = k-1
print(index[i][j])
print("==================================")
print(distance[i][j])
a = vec_lst_srcfile[i]
# print(a)
b = vec_lst_srcfile[index[i][j]]
# TEST
dist_squared = np.sum(np.square(a - b))
print(dist_squared)

50195
1.0689403
1.0689406


In [16]:
l05=l56=l67=l78=l89=l9_95=l95=l1 = 0
for i in range(len(vec_lst_srcfile)):
    num = float(distance[i][1])
    if num <0.5:
        l05 = l05 +1
    elif num >= 0.5 and num < 0.6:
        l56 = l56 +1
    elif num >= 0.6 and num < 0.7:
        l67 = l67 + 1
    elif num >= 0.7 and num < 0.8:
        l78 = l78 + 1
    elif num >= 0.8 and num < 0.9:
        l89 = l89 + 1
    elif num >= 0.9 and num < 0.95:
        l9_95 = l9_95 + 1
    elif num >= 0.95 and num < 1:
        l95 = l95 + 1
    elif num>=1:
        l1=l1+1
        
arrnum = [l05, l56, l67, l78, l89, l9_95, l95, l1]
t = len(lst_srcfile)/100
print(l05, l56, l67, l78, l89, l9_95, l95, l1)
print(l05/t, l56/t, l67/t, l78/t, l89/t, l9_95/t,l95/t, l1/t)
print(l05+l56+l67+l78+l89+l9_95+l95+l1)
print(len(lst_srcfile))


21170 14910 20225 20093 13604 3519 1954 1289
21.877971146294076 15.408623041627052 20.90136827745856 20.76495390847836 14.058947542474474 3.6366830639494028 2.0193460377826464 1.3321069819354305
96764
96764


In [18]:
start = time.time()
saperate = " || "

src_label = "S"
emb_label = "E"
thres_arr = [0.8, 0.85, 0.90, 0.95]
for i in range(len(thres_arr)):
    threshold = thres_arr[i]
    out_file_write = open("en_train_EM_"+str(thres_arr[i]), 'w')
    out_file_write_score = open("en_train_EM_score_"+str(thres_arr[i]), 'w')
    out_file_write_factor = open("en_train_EM_factor_"+str(thres_arr[i]), 'w')
    num_gth_thres = 0

    for i in range(len(lst_srcfile)):
        score = distance[i][1]
        content = ""
        content_label_arr = []
        if score >= threshold and score < 1:
            num_gth_thres += 1
            best_simi_index = index[i][1]
            content = lst_srcfile[i].strip() + saperate + lst_tarfile[best_simi_index].strip()

            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label.append(emb_label)
            content_label = content_label + [emb_label] * len(lst_tarfile[best_simi_index].split())

            content_label_arr = " ".join(content_label)
        else:
            content = lst_srcfile[i].strip()
            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label_arr = " ".join(content_label)
 
        out_file_write.writelines(content+ "\n")    
        out_file_write_factor.writelines(content_label_arr + "\n")
        out_file_write_score.writelines(str(score) + "\n")

    print('threshold =', threshold, "EM sentences =", num_gth_thres, "/", len(lst_srcfile), num_gth_thres/len(lst_srcfile), "%" )

    out_file_write.close()
    out_file_write_score.close()
    out_file_write_factor.close()

done = time.time()
elapsed = done - start
print('=== Ending, Total time (second): ', elapsed)

threshold = 0.8 EM sentences = 19077 / 96764 0.19714976644206522 %
threshold = 0.85 EM sentences = 11141 / 96764 0.11513579430366665 %
threshold = 0.9 EM sentences = 5473 / 96764 0.05656029101732049 %
threshold = 0.95 EM sentences = 1954 / 96764 0.020193460377826464 %
=== Ending, Total time (second):  5.346075534820557
